In [10]:
import sys
sys.path.append('../RecSysRep/')

In [11]:
import Basics.Load as ld

URM_all, _, _, _, _ = ld.getCOOs()
ICM_sel_7 = ld.getICMselected('7')
ICM_sel_9 = ld.getICMselected('9')
# URM_train, URM_val = ld.getSplit(URM_train_val, 5678, 0.8)
ICM_all = ld.getICMall()

In [12]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed=3456)
# URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
# evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13648 ( 0.0%) Users that have less than 1 test interactions


In [13]:
import os

ofp = "../models_temp/Similarity_Hybrid/"

models_to_combine_best = {
                            'RP3ICMnew': {'alpha': 1.029719677583138, 'beta': 1.0630164752134375, 'topK': 6964, 'normalize_similarity': True},
                            'RP3ICM' : {"topK": 2550, "alpha": 1.3058102610510849, "beta": 0.5150718337969987, "normalize_similarity": True, "implicit": True},
                            'IALS' : {"num_factors": 29, "epochs": 50, "confidence_scaling": "log", "alpha": 0.001, "epsilon": 0.001, "reg": 0.01},
                            'SLIMgensub': {"l1_ratio" : 0.025887359156206147, "topK": 2140, "alpha": 0.009567288586539689, "workers": 8, "mw": 1},
                            'SLIMBPR' : {"epochs": 440, "lambda_i": 0.007773815998802306, "lambda_j": 0.003342522366982381, "learning_rate": 0.010055161410725193, "topK": 4289, "random_seed": 1234, "sgd_mode": "sgd"},
                            'SLIMweig': {'l1_ratio': 0.0005247075138160404, 'topK': 4983, 'alpha': 0.06067400905430761, 'workers': 8, 'mw': 2.308619939318322},
                            'SLIMER': {'topK': 6000, 'l1_ratio': 0.0005495104968035837, 'alpha': 0.08007142704041009, 'workers': 8},
                            'P3alpha': {'topK': 4834, 'alpha': 1.764994849187595, 'normalize_similarity': True, 'implicit': True},
                            'RP3beta': {"topK": 1049, "alpha": 1.1626473723475605, "beta": 0.6765017195261293, "normalize_similarity": True, "implicit": True},
                            'sel7': {"shrink": 4000, "topK": 985, "feature_weighting": "TF-IDF", "normalize": True},
                            'sel9': {"shrink": 5212, "topK": 923, "feature_weighting": "TF-IDF", "normalize": True},
                            'sel3': {'shrink': 2211, 'topK': 188, 'feature_weighting': 'TF-IDF', 'normalize': True},
                            'ER_BPR': {'alpha': 0.58},
                            'TopPop': {},
                         }


In [14]:
def model_init(recommender, name, args):
    path_name = ofp + name + ".zip"
    print(path_name)
    if os.path.exists(path_name):
        print("Model found!")
        recommender.load_model(ofp, name)
    else:
        print("Model does not exists, creating...")
        if not os.path.exists(ofp):
            print("Main folder does not exist, creating...")
            os.makedirs(ofp)
        recommender.fit(**args)
        recommender.save_model(ofp, name)

In [15]:
from Recommenders.NonPersonalizedRecommender import TopPop
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommenderICM
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender

In [16]:
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridTwoRecommender
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridMultipleRecommender
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridOfHybridRecommender

In [9]:
# recommender1 = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train)
recommender2 = SLIM_BPR_Cython(URM_train)
# recommender3 = TopPop(URM_train)
# recommender3 = ItemKNNCBFRecommender(URM_train, ICM_sel_9)
# recommenderHybrid = ItemKNNScoresHybridTwoRecommender(URM_train, recommender1, recommender2)

# model_init(recommender1, 'SLIMER', models_to_combine_best['SLIMER'])
# model_init(recommender2, 'KNNweigh', models_to_combine_best['icm_weighted'])
model_init(recommender2, 'SLIMBPR_8888', models_to_combine_best['SLIMBPR'])
# model_init(recommender3, 'KNN_9', models_to_combine_best['sel9'])
# model_init(recommender3, 'TopPop', models_to_combine_best['TopPop'])


../models_temp/Similarity_Hybrid/SLIMBPR_8888.zip
Model does not exists, creating...
Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Automatic selection of fastest train mode. Unable to get current RAM status, you may be using a non-Linux operating system. Using dense matrix.
Processed 13650 (100.0%) in 0.44 sec. BPR loss is 1.09E+00. Sample per second: 31234
SLIM_BPR_Recommender: Epoch 1 of 440. Elapsed time 0.43 sec
Processed 13650 (100.0%) in 0.87 sec. BPR loss is 3.41E+00. Sample per second: 15634
SLIM_BPR_Recommender: Epoch 2 of 440. Elapsed time 0.86 sec
Processed 13650 (100.0%) in 1.31 sec. BPR loss is 5.29E+00. Sample per second: 10410
SLIM_BPR_Recommender: Epoch 3 of 440. Elapsed time 1.30 sec
Processed 13650 (100.0%) in 0.74 sec. BPR loss is 6.64E+00. Sample per second: 18438
SLIM_BPR_Recommender: Epoch 4 of 440. Elapsed time 1.73 sec
Processed 13650 (100.0%) in 1.17 sec. BPR loss is 8.10E+00. Sample per second: 11643
SLIM_BPR_Recommender: Epoch 5 

Processed 13650 (100.0%) in 0.91 sec. BPR loss is 2.29E+01. Sample per second: 14926
SLIM_BPR_Recommender: Epoch 55 of 440. Elapsed time 23.91 sec
Processed 13650 (100.0%) in 1.39 sec. BPR loss is 2.24E+01. Sample per second: 9844
SLIM_BPR_Recommender: Epoch 56 of 440. Elapsed time 24.38 sec
Processed 13650 (100.0%) in 0.82 sec. BPR loss is 2.31E+01. Sample per second: 16713
SLIM_BPR_Recommender: Epoch 57 of 440. Elapsed time 24.81 sec
Processed 13650 (100.0%) in 1.26 sec. BPR loss is 2.41E+01. Sample per second: 10843
SLIM_BPR_Recommender: Epoch 58 of 440. Elapsed time 25.25 sec
Processed 13650 (100.0%) in 0.69 sec. BPR loss is 2.39E+01. Sample per second: 19642
SLIM_BPR_Recommender: Epoch 59 of 440. Elapsed time 25.68 sec
Processed 13650 (100.0%) in 1.13 sec. BPR loss is 2.36E+01. Sample per second: 12122
SLIM_BPR_Recommender: Epoch 60 of 440. Elapsed time 26.12 sec
Processed 13650 (100.0%) in 0.56 sec. BPR loss is 2.37E+01. Sample per second: 24540
SLIM_BPR_Recommender: Epoch 61 of 

Processed 13650 (100.0%) in 1.03 sec. BPR loss is 2.73E+01. Sample per second: 13305
SLIM_BPR_Recommender: Epoch 111 of 440. Elapsed time 48.02 sec
Processed 13650 (100.0%) in 0.46 sec. BPR loss is 2.72E+01. Sample per second: 29103
SLIM_BPR_Recommender: Epoch 112 of 440. Elapsed time 48.46 sec
Processed 13650 (100.0%) in 0.90 sec. BPR loss is 2.74E+01. Sample per second: 15131
SLIM_BPR_Recommender: Epoch 113 of 440. Elapsed time 48.89 sec
Processed 13650 (100.0%) in 1.33 sec. BPR loss is 2.68E+01. Sample per second: 10246
SLIM_BPR_Recommender: Epoch 114 of 440. Elapsed time 49.32 sec
Processed 13650 (100.0%) in 0.76 sec. BPR loss is 2.73E+01. Sample per second: 17950
SLIM_BPR_Recommender: Epoch 115 of 440. Elapsed time 49.75 sec
Processed 13650 (100.0%) in 1.19 sec. BPR loss is 2.67E+01. Sample per second: 11464
SLIM_BPR_Recommender: Epoch 116 of 440. Elapsed time 50.18 sec
Processed 13650 (100.0%) in 0.62 sec. BPR loss is 2.71E+01. Sample per second: 21916
SLIM_BPR_Recommender: Epoch

SLIM_BPR_Recommender: Epoch 166 of 440. Elapsed time 1.20 min
Processed 13650 (100.0%) in 1.23 sec. BPR loss is 2.93E+01. Sample per second: 11103
SLIM_BPR_Recommender: Epoch 167 of 440. Elapsed time 1.20 min
Processed 13650 (100.0%) in 0.66 sec. BPR loss is 2.93E+01. Sample per second: 20574
SLIM_BPR_Recommender: Epoch 168 of 440. Elapsed time 1.21 min
Processed 13650 (100.0%) in 1.09 sec. BPR loss is 2.90E+01. Sample per second: 12482
SLIM_BPR_Recommender: Epoch 169 of 440. Elapsed time 1.22 min
Processed 13650 (100.0%) in 0.53 sec. BPR loss is 2.94E+01. Sample per second: 25868
SLIM_BPR_Recommender: Epoch 170 of 440. Elapsed time 1.23 min
Processed 13650 (100.0%) in 0.96 sec. BPR loss is 2.98E+01. Sample per second: 14207
SLIM_BPR_Recommender: Epoch 171 of 440. Elapsed time 1.23 min
Processed 13650 (100.0%) in 1.39 sec. BPR loss is 2.99E+01. Sample per second: 9779
SLIM_BPR_Recommender: Epoch 172 of 440. Elapsed time 1.24 min
Processed 13650 (100.0%) in 0.83 sec. BPR loss is 2.96E+0

SLIM_BPR_Recommender: Epoch 222 of 440. Elapsed time 1.60 min
Processed 13650 (100.0%) in 0.72 sec. BPR loss is 3.03E+01. Sample per second: 18964
SLIM_BPR_Recommender: Epoch 223 of 440. Elapsed time 1.61 min
Processed 13650 (100.0%) in 1.15 sec. BPR loss is 2.99E+01. Sample per second: 11881
SLIM_BPR_Recommender: Epoch 224 of 440. Elapsed time 1.62 min
Processed 13650 (100.0%) in 0.59 sec. BPR loss is 3.05E+01. Sample per second: 23176
SLIM_BPR_Recommender: Epoch 225 of 440. Elapsed time 1.63 min
Processed 13650 (100.0%) in 1.05 sec. BPR loss is 3.01E+01. Sample per second: 13061
SLIM_BPR_Recommender: Epoch 226 of 440. Elapsed time 1.63 min
Processed 13650 (100.0%) in 0.51 sec. BPR loss is 3.06E+01. Sample per second: 26967
SLIM_BPR_Recommender: Epoch 227 of 440. Elapsed time 1.64 min
Processed 13650 (100.0%) in 0.99 sec. BPR loss is 3.13E+01. Sample per second: 13756
SLIM_BPR_Recommender: Epoch 228 of 440. Elapsed time 1.65 min
Processed 13650 (100.0%) in 1.48 sec. BPR loss is 3.09E+

SLIM_BPR_Recommender: Epoch 278 of 440. Elapsed time 2.01 min
Processed 13650 (100.0%) in 1.11 sec. BPR loss is 3.23E+01. Sample per second: 12272
SLIM_BPR_Recommender: Epoch 279 of 440. Elapsed time 2.02 min
Processed 13650 (100.0%) in 0.54 sec. BPR loss is 3.13E+01. Sample per second: 25214
SLIM_BPR_Recommender: Epoch 280 of 440. Elapsed time 2.03 min
Processed 13650 (100.0%) in 0.97 sec. BPR loss is 3.15E+01. Sample per second: 14066
SLIM_BPR_Recommender: Epoch 281 of 440. Elapsed time 2.03 min
Processed 13650 (100.0%) in 1.40 sec. BPR loss is 3.14E+01. Sample per second: 9739
SLIM_BPR_Recommender: Epoch 282 of 440. Elapsed time 2.04 min
Processed 13650 (100.0%) in 0.83 sec. BPR loss is 3.20E+01. Sample per second: 16374
SLIM_BPR_Recommender: Epoch 283 of 440. Elapsed time 2.05 min
Processed 13650 (100.0%) in 1.26 sec. BPR loss is 3.16E+01. Sample per second: 10793
SLIM_BPR_Recommender: Epoch 284 of 440. Elapsed time 2.05 min
Processed 13650 (100.0%) in 0.70 sec. BPR loss is 3.11E+0

SLIM_BPR_Recommender: Epoch 334 of 440. Elapsed time 2.41 min
Processed 13650 (100.0%) in 1.30 sec. BPR loss is 3.28E+01. Sample per second: 10493
SLIM_BPR_Recommender: Epoch 335 of 440. Elapsed time 2.42 min
Processed 13650 (100.0%) in 0.73 sec. BPR loss is 3.26E+01. Sample per second: 18650
SLIM_BPR_Recommender: Epoch 336 of 440. Elapsed time 2.43 min
Processed 13650 (100.0%) in 1.17 sec. BPR loss is 3.26E+01. Sample per second: 11717
SLIM_BPR_Recommender: Epoch 337 of 440. Elapsed time 2.44 min
Processed 13650 (100.0%) in 0.60 sec. BPR loss is 3.37E+01. Sample per second: 22899
SLIM_BPR_Recommender: Epoch 338 of 440. Elapsed time 2.44 min
Processed 13650 (100.0%) in 1.03 sec. BPR loss is 3.29E+01. Sample per second: 13224
SLIM_BPR_Recommender: Epoch 339 of 440. Elapsed time 2.45 min
Processed 13650 (100.0%) in 0.47 sec. BPR loss is 3.25E+01. Sample per second: 29336
SLIM_BPR_Recommender: Epoch 340 of 440. Elapsed time 2.46 min
Processed 13650 (100.0%) in 0.90 sec. BPR loss is 3.30E+

SLIM_BPR_Recommender: Epoch 390 of 440. Elapsed time 2.82 min
Processed 13650 (100.0%) in 0.47 sec. BPR loss is 3.29E+01. Sample per second: 29214
SLIM_BPR_Recommender: Epoch 391 of 440. Elapsed time 2.82 min
Processed 13650 (100.0%) in 0.90 sec. BPR loss is 3.37E+01. Sample per second: 15161
SLIM_BPR_Recommender: Epoch 392 of 440. Elapsed time 2.83 min
Processed 13650 (100.0%) in 1.34 sec. BPR loss is 3.34E+01. Sample per second: 10214
SLIM_BPR_Recommender: Epoch 393 of 440. Elapsed time 2.84 min
Processed 13650 (100.0%) in 0.77 sec. BPR loss is 3.32E+01. Sample per second: 17738
SLIM_BPR_Recommender: Epoch 394 of 440. Elapsed time 2.85 min
Processed 13650 (100.0%) in 1.20 sec. BPR loss is 3.40E+01. Sample per second: 11369
SLIM_BPR_Recommender: Epoch 395 of 440. Elapsed time 2.85 min
Processed 13650 (100.0%) in 0.63 sec. BPR loss is 3.32E+01. Sample per second: 21540
SLIM_BPR_Recommender: Epoch 396 of 440. Elapsed time 2.86 min
Processed 13650 (100.0%) in 1.06 sec. BPR loss is 3.30E+

In [ ]:

recommenderHybrid.fit(alpha = 0.58)


In [17]:
def rank_models(evaluator, recommenders):
    for r in recommenders:
        r_d, _ = evaluator.evaluateRecommender(r)
        print(r.RECOMMENDER_NAME, r_d.loc[10]['MAP'])

In [ ]:
rank_models(evaluator_validation, [recommender1, recommender2])

In [ ]:
import numpy as np

# TWO RECOMMENDERS

def test_percentage(recommender_1, recommender_2, evaluator, step):
    recommender = ItemKNNScoresHybridOfHybridRecommender(URM_train, recommender_1, recommender_2)
    results = []
    alp_space = np.linspace(0.88, 0.99, step, True)
    for alp in alp_space:
        recommender.fit(alpha = alp)
        r_d, _ = evaluator.evaluateRecommender(recommender)
        print(alp, ":", r_d.loc[10]['MAP'])
        results.append(r_d.loc[10]['MAP'])
    
    return alp_space, results

In [ ]:
alp_space, results = test_percentage(recommenderHybrid, recommender3, evaluator_validation, 5)

In [ ]:
import numpy as np

# THREE RECOMMENDERS

def test_percentage(recommender_1, recommender_2, recommender_3, evaluator, step):
    recommender = ItemKNNScoresHybridMultipleRecommender(URM_train, recommender_1, recommender_2, recommender_3)
    results = []
    alp_space = np.linspace(12, 18, step, True)
    for alp in alp_space:
        recommender.fit(alpha = alp, beta = 5, gamma = 1)
        r_d, _ = evaluator.evaluateRecommender(recommender)
        print(alp, ":", r_d.loc[10]['MAP'])
        results.append(r_d.loc[10]['MAP'])
    
    return alp_space, results

In [ ]:
alp_space, results = test_percentage(recommender1, recommender2, recommender3, evaluator_validation, 5)

In [ ]:
import matplotlib.pyplot as plt

_ = plt.figure(figsize=(16, 9))
plt.plot(alp_space,results, label='MAP variability')
plt.ylabel('MAP')
plt.xlabel('User Group')
plt.legend()
plt.show()

plt.savefig('userwise.png')

* # SLIMER + KNNweight: 0.2499 with alpha = 0.95
* # SLIMER + SLIMBPR: 0.2507 with alpha = 0.58
* # (SLIMER + SLIMBPR) + KNN_selected_3: 0.2509 with alpha 0.96

In [ ]:
from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {
    "shrink": Integer(10, 500),
    "topK": Integer(200, 1000),
    "feature_weighting": Categorical(["none", "TF-IDF"]),
    "normalize": Categorical([True, False]),
    "ICM_weight": Integer(5, 500),
}

In [ ]:
from Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

recommender_class = ItemKNN_CFCBF_Hybrid_Recommender

hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                         evaluator_validation=evaluator_validation)

In [ ]:
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
  
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train, ICM_genre_all],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}, # Additiona hyperparameters for the fit function
)

In [ ]:
if __name__ == '__main__':
    hyperparameterSearch.search(recommender_input_args,
                           hyperparameter_search_space = hyperparameters_range_dictionary,
                           n_cases = n_cases,
                           n_random_starts = n_random_starts,
                           output_folder_path = output_folder_path,
                           output_file_name_root = recommender_class.RECOMMENDER_NAME,
                           metric_to_optimize = metric_to_optimize,
                           cutoff_to_optimize = cutoff_to_optimize,
                          )

In [ ]:

recommender1 = MultiThreadSLIM_SLIMElasticNetRecommender(URM_all)
# recommender1.fit(**models_to_combine_best['SLIMER'])
# recommender1.save_model(ofp + 'SLIMER_ALL/', 'SLIMER_ALL')
recommender1.load_model(ofp + 'SLIMER_ALL/', 'SLIMER_ALL')

recommender2 = SLIM_BPR_Cython(URM_all)
# recommender2.fit(**models_to_combine_best['SLIMBPR'])
# recommender2.save_model(ofp + 'SLIMBPR_ALL/', 'SLIMBPR_ALL')
recommender2.load_model(ofp + 'SLIMBPR_ALL/', 'SLIMBPR_ALL')

recommenderHybrid = ItemKNNScoresHybridTwoRecommender(URM_all, recommender1, recommender2)
recommenderHybrid.fit(alpha = 0.58)

recommender = recommenderHybrid

import pandas as pd
at = 10

user_test_path = '../data/data_target_users_test.csv'
user_test_dataframe = pd.read_csv(filepath_or_buffer=user_test_path,
sep=",",
dtype={0:int})

subm_set = user_test_dataframe.to_numpy().T[0]


subm_res = {"user_id":[], "item_list":[]}

for user_id in subm_set:
	subm_res["user_id"].append(user_id)
	res = recommender.recommend(user_id, cutoff=at)
	res = ' '.join(map(str, res))
	if user_id < 3:
		print(user_id)
		print(res)
	subm_res["item_list"].append(res)


	# print(subm_res)

submission = pd.DataFrame.from_dict(subm_res)
	# submission

from datetime import datetime

now = datetime.now() # current date and time


submission.to_csv('../subs/submission {:%Y_%m_%d %H_%M_%S}.csv'.format(now), index=False)



In [ ]:
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
CFrecommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train)
CFrecommender.load_model('../result_experiments/BEST_SLIM_PURECF/', file_name = CFrecommender.RECOMMENDER_NAME + "_my_own_save.zip")

from Recommenders.FeatureWeighting.CFW_D_Similarity_Linalg import CFW_D_Similarity_Linalg
FW_recommender = CFW_D_Similarity_Linalg(URM_train, ICM_stacked, CFrecommender.W_sparse)
FW_recommender.load_model('../result_experiments/BEST_FW/', file_name = FW_recommender.RECOMMENDER_NAME + "_my_own_save.zip")

In [ ]:
import scipy.sparse as sps
ICM_length_all_5km = ld.getICMlength('5km')
ICM_stacked = sps.hstack((ICM_genre_all, ICM_subgenre_all, ICM_channel_all, ICM_length_all_5km))

In [ ]:
import pandas as pd
argsort_features = np.argsort(-FW_recommender.D_best)

selection_quota = 0.1
        
n_to_select = int(selection_quota*len(argsort_features))
selected_features = argsort_features[:n_to_select]
        
ICM_selected = ICM_stacked.todense()[:,selected_features]
pd.DataFrame(ICM_selected).to_csv('ICM_selected_1.csv', index=False, header=True)


In [ ]:
import scipy.sparse as sps
import pandas as pd
URM_all, ICM_genre_all, ICM_subgenre_all, ICM_channel_all, _ = ld.getCOOs()
ICM_length_all_5km = ld.getICMlength('5km')
pd.DataFrame(sps.hstack([ICM_genre_all, ICM_subgenre_all, ICM_channel_all, ICM_length_all_5km]).todense()).to_csv('ICM_ALL.csv', index=False, header=True)

In [19]:
recommender1 = recommender1 = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train)
model_init(recommender1, 'SLIMER', models_to_combine_best['SLIMER'])
rank_models(evaluator_validation, [recommender1])
recommender1.fit(**models_to_combine_best['SLIMER'])
rank_models(evaluator_validation, [recommender1])

# recommender1.save_model(ofp + 'SLIMER_ALL/', 'SLIMER_ALL')
# recommender1.load_model(ofp + 'SLIMER_ALL/', 'SLIMER_ALL')

../models_temp/Similarity_Hybrid/SLIMER.zip
Model found!
SLIMElasticNetRecommender: Loading model from file '../models_temp/Similarity_Hybrid/SLIMER'
SLIMElasticNetRecommender: Loading complete


  1%|█                                                                             | 256/18059 [00:30<07:37, 38.91it/s]

EvaluatorHoldout: Processed 13648 (100.0%) in 13.65 sec. Users per second: 1000



  0%|                                                                                        | 0/18059 [00:00<?, ?it/s]

SLIMElasticNetRecommender 0.24901781934442407


...

 74%|███████████████████████████████████████████████████████▉                    | 13288/18059 [10:53<04:45, 16.72it/s]


 86%|█████████████████████████████████████████████████████████████████▏          | 15496/18059 [12:41<01:35, 26.96it/s]


 98%|██████████████████████████████████████████████████████████████████████████▎ | 17672/18059 [14:32<00:19, 19.53it/s]


100%|███████████████████████████████████████████████████████████████████████████▉| 18056/18059 [15:00<00:00, 47.87it/s]

EvaluatorHoldout: Processed 13648 (100.0%) in 13.89 sec. Users per second: 982
SLIMElasticNetRecommender 0.24901764779768892
